# Multi-Agent Systems: Complete Guide with Theory + Code**Focus:** CrewAI, AutoGen, and Multi-Agent Collaboration Patterns  **Topics:** Role-based agents, communication protocols, coordination strategies## OverviewThis comprehensive notebook explores multi-agent systems where multiple AI agents collaborate to solve complex problems.### What You'll Learn:1. **Multi-Agent Fundamentals** - Architecture, communication, coordination2. **CrewAI Framework** - Role-based agent collaboration3. **AutoGen Framework** - Conversational multi-agent patterns4. **Collaboration Patterns** - Hierarchical, sequential, parallel, consensus5. **Production Patterns** - Scaling, monitoring, orchestration6. **Real-World Applications** - Software development, research, content creation### Frameworks Covered:- **CrewAI**: Role-based agents with hierarchical management- **AutoGen**: Conversational agents with code execution- **LangGraph**: Custom multi-agent orchestration

## Multi-Agent Systems: Foundational Theory### What is a Multi-Agent System?A **multi-agent system (MAS)** consists of multiple autonomous agents that interact to achieve individual or collective goals.```mermaidgraph TD    A[Complex Task] --> B[Agent 1:<br/>Researcher]    A --> C[Agent 2:<br/>Writer]    A --> D[Agent 3:<br/>Critic]        B -->|Findings| E[Shared Context]    C -->|Draft| E    D -->|Feedback| E        E --> F[Coordinator]    F --> G[Final Output]        style A fill:#e1f5ff    style B fill:#fff9c4    style C fill:#fff9c4    style D fill:#fff9c4    style E fill:#ffe0b2    style F fill:#ffccbc    style G fill:#c8e6c9```### Key Characteristics| Aspect | Single Agent | Multi-Agent System ||--------|--------------|-------------------|| **Complexity** | Limited by single LLM | Distributed across agents || **Specialization** | Generalist | Each agent specialized || **Scalability** | Hard to scale | Easier to add agents || **Robustness** | Single point of failure | Redundancy possible || **Cost** | Lower | Higher (multiple LLMs) || **Coordination** | Not needed | Critical challenge |

### Multi-Agent Architecture Patterns```mermaidgraph TB    subgraph "1. Hierarchical (Manager-Worker)"        H1[Manager Agent] --> H2[Worker 1]        H1 --> H3[Worker 2]        H1 --> H4[Worker 3]    end        subgraph "2. Sequential Pipeline"        S1[Agent 1] --> S2[Agent 2]        S2 --> S3[Agent 3]        S3 --> S4[Agent 4]    end        subgraph "3. Collaborative Network"        C1[Agent 1] <--> C2[Agent 2]        C2 <--> C3[Agent 3]        C3 <--> C1    end        subgraph "4. Competitive/Debate"        D1[Agent 1] -->|Proposal| D4[Mediator]        D2[Agent 2] -->|Counter| D4        D3[Agent 3] -->|Alternative| D4        D4 --> D5[Best Solution]    end```### Communication Protocols```mermaidsequenceDiagram    participant Manager    participant Agent1    participant Agent2    participant SharedMemory        Manager->>Agent1: Task Assignment    Agent1->>SharedMemory: Write findings    Manager->>Agent2: Task Assignment    Agent2->>SharedMemory: Read findings    Agent2->>SharedMemory: Write analysis    SharedMemory-->>Manager: Aggregated results    Manager->>Manager: Synthesize final output        Note over Manager,SharedMemory: Async communication via shared state```

## Setup

In [ ]:
# Install required packages!pip install -q crewai anthropic openai langchain langgraph python-dotenv!pip install -q pyautogen

In [ ]:
import osfrom dotenv import load_dotenvimport jsonfrom typing import List, Dict, Anyfrom datetime import datetime# CrewAI importsfrom crewai import Agent, Task, Crew, Processfrom langchain_anthropic import ChatAnthropicfrom langchain_openai import ChatOpenAI# Load environmentload_dotenv()# Initialize LLMsclaude = ChatAnthropic(    model="claude-3-5-sonnet-20241022",    api_key=os.getenv("ANTHROPIC_API_KEY"),    temperature=0.7)gpt = ChatOpenAI(    model="gpt-4o",    api_key=os.getenv("OPENAI_API_KEY"),    temperature=0.7)print("✓ Setup complete!")print("✓ CrewAI and AutoGen ready")

---# Part 1: CrewAI - Role-Based Multi-Agent Framework## Theory**CrewAI** uses a role-based approach where agents have specific roles, goals, and backstories.### Core Concepts1. **Agent**: Autonomous unit with role, goal, backstory2. **Task**: Specific work assigned to agent(s)3. **Crew**: Collection of agents working together4. **Process**: How agents collaborate (sequential, hierarchical, consensus)### CrewAI Architecture```mermaidgraph TD    A[Crew Definition] --> B[Agent 1:<br/>Role + Goal]    A --> C[Agent 2:<br/>Role + Goal]    A --> D[Agent 3:<br/>Role + Goal]        E[Task 1] --> B    F[Task 2] --> C    G[Task 3] --> D        B --> H[Process Manager]    C --> H    D --> H        H --> I{Process Type}    I -->|Sequential| J[One after another]    I -->|Hierarchical| K[Manager delegates]    I -->|Consensus| L[Vote on decisions]        J --> M[Final Output]    K --> M    L --> M        style A fill:#e1f5ff    style H fill:#ffe0b2    style I fill:#fff9c4    style M fill:#c8e6c9```### Agent Properties| Property | Purpose | Example ||----------|---------|---------|| **Role** | Agent's job title | "Senior Researcher" || **Goal** | What agent aims to achieve | "Find accurate information" || **Backstory** | Agent's context/expertise | "PhD in Computer Science..." || **Tools** | Functions agent can use | [search_tool, calculator] || **Verbose** | Logging detail | True/False || **Allow Delegation** | Can assign to others | True/False |

## Implementation: CrewAI Research TeamLet's create a research team with 3 specialized agents.

In [ ]:
from crewai import Agent, Task, Crew, Processfrom crewai_tools import tool# Define custom tools@tool("Search Tool")def search_web(query: str) -> str:    """Search the web for information."""    # Simulated search results    search_db = {        "ai agents": "AI agents are autonomous systems that use LLMs to make decisions and take actions.",        "crewai": "CrewAI is a framework for orchestrating role-playing AI agents.",        "multi-agent": "Multi-agent systems involve multiple AI agents collaborating to solve problems."    }    for key, value in search_db.items():        if key in query.lower():            return value    return f"Search results for '{query}': General information about the topic."@tool("Analysis Tool")def analyze_data(data: str) -> str:    """Analyze and synthesize information."""    return f"Analysis complete: {len(data)} characters processed. Key themes extracted."print("✓ Tools defined")

In [ ]:
# Create specialized agentsresearcher = Agent(    role="Senior Research Analyst",    goal="Discover cutting-edge developments in AI agents",    backstory="""You are an expert research analyst with a PhD in Computer Science.    You excel at finding accurate, relevant information and identifying key trends.    Your analysis is always thorough and well-cited.""",    tools=[search_web],    verbose=True,    llm=claude)writer = Agent(    role="Technical Content Writer",    goal="Create engaging and accurate technical content",    backstory="""You are a skilled technical writer who excels at making complex    topics accessible. You write clear, structured content with compelling narratives.    Your writing is always well-organized and professionally formatted.""",    tools=[analyze_data],    verbose=True,    llm=claude)editor = Agent(    role="Senior Editor",    goal="Ensure content quality and accuracy",    backstory="""You are a meticulous editor with 15 years of experience.    You catch errors, improve clarity, and ensure content meets high standards.    Your feedback is constructive and actionable.""",    verbose=True,    llm=claude)print("✓ Created 3 specialized agents")print(f"  - {researcher.role}")print(f"  - {writer.role}")print(f"  - {editor.role}")

In [ ]:
# Define tasks for each agentresearch_task = Task(    description="""Research the latest developments in multi-agent AI systems.    Focus on:    1. Recent frameworks (CrewAI, AutoGen)    2. Key use cases and applications    3. Challenges and best practices        Provide a comprehensive summary with key findings.""",    agent=researcher,    expected_output="Detailed research summary with key findings")writing_task = Task(    description="""Using the research findings, write a 500-word article about    multi-agent AI systems. Include:    1. Introduction explaining the concept    2. Key frameworks and their differences    3. Real-world applications    4. Conclusion with future outlook        Make it engaging and accessible to technical readers.""",    agent=writer,    expected_output="Well-structured 500-word article")editing_task = Task(    description="""Review and edit the article for:    1. Technical accuracy    2. Clarity and flow    3. Grammar and style    4. Overall quality        Provide the final polished version.""",    agent=editor,    expected_output="Final polished article ready for publication")print("✓ Created 3 tasks")

In [ ]:
# Create crew and executeprint("=" * 70)print("CREWAI: Sequential Research Team")print("=" * 70)crew = Crew(    agents=[researcher, writer, editor],    tasks=[research_task, writing_task, editing_task],    process=Process.sequential,  # Execute tasks in order    verbose=True)print("\nExecuting crew workflow...\n")result = crew.kickoff()print("\n" + "=" * 70)print("FINAL OUTPUT")print("=" * 70)print(result)print("=" * 70)

### CrewAI Process Types```mermaidgraph LR    subgraph "Sequential Process"        S1[Task 1] --> S2[Task 2]        S2 --> S3[Task 3]        S3 --> S4[Output]    end        subgraph "Hierarchical Process"        H1[Manager] --> H2[Task 1]        H1 --> H3[Task 2]        H1 --> H4[Task 3]        H2 --> H5[Synthesis]        H3 --> H5        H4 --> H5    end        subgraph "Consensus Process"        C1[Agent 1] -->|Vote| C4[Decision]        C2[Agent 2] -->|Vote| C4        C3[Agent 3] -->|Vote| C4        C4 --> C5[Output]    end        style S1 fill:#e1f5ff    style H1 fill:#fff9c4    style C4 fill:#ffe0b2```

---# Part 2: AutoGen - Conversational Multi-Agent Framework## Theory**AutoGen** by Microsoft enables conversational agents that can generate and execute code.### Key Features- Conversational agent interactions- Code generation and execution- Human-in-the-loop support- Multi-turn conversations### AutoGen Architecture```mermaidgraph TD    A[User Proxy<br/>Human Interface] <--> B[Assistant<br/>Code Generator]    B <--> C[Code Executor]    C -->|Results| B    B <--> D[Other Agents]    D <--> B        style A fill:#e1f5ff    style B fill:#fff9c4    style C fill:#ffe0b2    style D fill:#e1f5ff```

In [ ]:
# AutoGen Implementationimport autogen# Configure LLMconfig_list = [{    "model": "gpt-4o",    "api_key": os.getenv("OPENAI_API_KEY")}]llm_config = {    "config_list": config_list,    "temperature": 0.7}# Create assistant agentassistant = autogen.AssistantAgent(    name="DataAnalyst",    system_message="""You are a data analyst expert. You help analyze data    and provide insights. You can write Python code to process data.""",    llm_config=llm_config)# Create user proxy (executes code)user_proxy = autogen.UserProxyAgent(    name="UserProxy",    human_input_mode="NEVER",  # Fully automated    max_consecutive_auto_reply=5,    code_execution_config={        "work_dir": "coding",        "use_docker": False    })print("✓ AutoGen agents created")

In [ ]:
# Test AutoGen conversationprint("=" * 70)print("AUTOGEN: Data Analysis Task")print("=" * 70)task = """Analyze this dataset: [10, 25, 30, 15, 40, 35, 20]Calculate the mean, median, and standard deviation.Then create a simple visualization recommendation."""print(f"\nTask: {task}\n")# Initiate conversationuser_proxy.initiate_chat(    assistant,    message=task)print("\n" + "=" * 70)

---# Part 3: Collaboration Patterns## Pattern 1: Hierarchical (Manager-Worker)```mermaidgraph TD    M[Manager Agent] --> W1[Worker 1:<br/>Research]    M --> W2[Worker 2:<br/>Analysis]    M --> W3[Worker 3:<br/>Synthesis]        W1 -->|Report| M    W2 -->|Report| M    W3 -->|Report| M        M --> F[Final Decision]        style M fill:#ffccbc    style W1 fill:#e1f5ff    style W2 fill:#e1f5ff    style W3 fill:#e1f5ff    style F fill:#c8e6c9```

In [ ]:
# Hierarchical pattern with CrewAImanager = Agent(    role="Project Manager",    goal="Coordinate team and ensure project success",    backstory="Experienced PM with 10 years managing technical teams",    verbose=True,    llm=claude,    allow_delegation=True  # Can delegate to other agents)# Create hierarchical crewhierarchical_crew = Crew(    agents=[manager, researcher, writer],    tasks=[research_task, writing_task],    process=Process.hierarchical,  # Manager coordinates    verbose=True,    manager_llm=claude)print("✓ Hierarchical crew created with manager")

## Pattern 2: Sequential Pipeline```mermaidgraph LR    A[Input] --> B[Agent 1:<br/>Collect]    B -->|Data| C[Agent 2:<br/>Process]    C -->|Insights| D[Agent 3:<br/>Report]    D -->|Final| E[Output]        style A fill:#e1f5ff    style B fill:#fff9c4    style C fill:#fff9c4    style D fill:#fff9c4    style E fill:#c8e6c9```**Use When:** Clear pipeline with dependencies between stages

## Pattern 3: Debate/Competition```mermaidgraph TD    T[Topic] --> A1[Agent 1:<br/>For Position]    T --> A2[Agent 2:<br/>Against Position]    T --> A3[Agent 3:<br/>Neutral Analysis]        A1 -->|Argument| J[Judge Agent]    A2 -->|Counter| J    A3 -->|Analysis| J        J --> D[Best Solution]        style T fill:#e1f5ff    style A1 fill:#c8e6c9    style A2 fill:#ffccbc    style A3 fill:#fff9c4    style J fill:#ffe0b2    style D fill:#c8e6c9```**Use When:** Need diverse perspectives or want to challenge assumptions

---# Part 4: Production Best Practices## Multi-Agent System Design Principles### 1. Agent Specialization```mermaidgraph TD    A[Complex Problem] --> B{Decompose}    B --> C[Subtask 1:<br/>Research]    B --> D[Subtask 2:<br/>Analysis]    B --> E[Subtask 3:<br/>Writing]        C --> F[Specialized<br/>Agent 1]    D --> G[Specialized<br/>Agent 2]    E --> H[Specialized<br/>Agent 3]        F --> I[Integration]    G --> I    H --> I        I --> J[Solution]        style A fill:#e1f5ff    style F fill:#fff9c4    style G fill:#fff9c4    style H fill:#fff9c4    style I fill:#ffe0b2    style J fill:#c8e6c9```**Principle:** Each agent should have a clear, focused role rather than being a generalist.### 2. Communication Overhead| # Agents | Communication Paths | Complexity ||----------|-------------------|------------|| 2 | 1 | Low || 3 | 3 | Medium || 4 | 6 | High || 5 | 10 | Very High || n | n(n-1)/2 | O(n²) |**Principle:** Keep agent count reasonable (3-5) to manage communication complexity.

### 3. Error Handling in Multi-Agent Systems```mermaidgraph TD    A[Agent Execution] --> B{Success?}    B -->|Yes| C[Continue]    B -->|No| D{Retry?}    D -->|Yes| E[Retry with<br/>Different Agent]    D -->|No| F[Escalate to<br/>Human]    E --> A    F --> G[Human Decision]    G --> H[Resume or Abort]        style A fill:#e1f5ff    style B fill:#fff9c4    style D fill:#fff9c4    style F fill:#ffccbc    style C fill:#c8e6c9```

### 4. Monitoring Multi-Agent Systems**Key Metrics:**1. **Agent Performance**   - Task completion rate per agent   - Average execution time per agent   - Error rate per agent2. **System Performance**   - End-to-end latency   - Total cost (all agent calls)   - Success rate3. **Communication Metrics**   - Number of agent interactions   - Message passing overhead   - Coordination time4. **Quality Metrics**   - Output quality scores   - Human feedback ratings   - Consistency across agents

In [ ]:
# Monitoring wrapper for agentsclass MonitoredCrew:    def __init__(self, crew):        self.crew = crew        self.metrics = {            "total_time": 0,            "agent_times": {},            "task_count": 0,            "errors": []        }        def execute(self):        import time        start = time.time()                try:            result = self.crew.kickoff()            self.metrics["total_time"] = time.time() - start            self.metrics["task_count"] = len(self.crew.tasks)            return result        except Exception as e:            self.metrics["errors"].append(str(e))            raise        def get_metrics(self):        return {            "total_time": f"{self.metrics['total_time']:.2f}s",            "tasks_completed": self.metrics["task_count"],            "avg_time_per_task": f"{self.metrics['total_time']/max(self.metrics['task_count'],1):.2f}s",            "errors": len(self.metrics["errors"])        }# Example usagemonitored_crew = MonitoredCrew(crew)# result = monitored_crew.execute()# print(f"Metrics: {monitored_crew.get_metrics()}")print("✓ Monitoring system ready")

---# Part 5: Real-World Applications## Use Case 1: Software Development Team```mermaidgraph TD    PM[Product Manager] --> BA[Business Analyst]    PM --> DEV[Developer]    PM --> QA[QA Engineer]        BA -->|Requirements| DEV    DEV -->|Code| QA    QA -->|Bugs| DEV    QA -->|Approved| PM        style PM fill:#ffccbc    style BA fill:#e1f5ff    style DEV fill:#fff9c4    style QA fill:#c8e6c9```

In [ ]:
# Software development crewproduct_manager = Agent(    role="Product Manager",    goal="Define clear requirements and acceptance criteria",    backstory="Experienced PM with strong technical background",    llm=claude)developer = Agent(    role="Senior Developer",    goal="Write clean, efficient code that meets requirements",    backstory="Full-stack developer with 10 years experience",    llm=claude)qa_engineer = Agent(    role="QA Engineer",    goal="Ensure code quality and find potential issues",    backstory="Detail-oriented QA with automation expertise",    llm=claude)# Tasksrequirements_task = Task(    description="Define requirements for a user authentication feature",    agent=product_manager,    expected_output="Detailed requirements document")development_task = Task(    description="Design the authentication system architecture",    agent=developer,    expected_output="Technical architecture document")qa_task = Task(    description="Create test plan for authentication feature",    agent=qa_engineer,    expected_output="Comprehensive test plan")dev_crew = Crew(    agents=[product_manager, developer, qa_engineer],    tasks=[requirements_task, development_task, qa_task],    process=Process.sequential)print("✓ Software development crew ready")

## Use Case 2: Content Creation Pipeline```mermaidgraph LR    R[Researcher] -->|Facts| W[Writer]    W -->|Draft| E[Editor]    E -->|Feedback| W    E -->|Final| S[SEO Specialist]    S -->|Optimized| P[Publisher]        style R fill:#e1f5ff    style W fill:#fff9c4    style E fill:#ffe0b2    style S fill:#fff9c4    style P fill:#c8e6c9```

---# Part 6: Interview Q&A## Common Questions**Q1: When should you use multi-agent systems vs single agent?****Answer:**Use **Multi-Agent** when:- Task requires different specializations- Need parallel processing- Want redundancy/voting- Problem naturally decomposesUse **Single Agent** when:- Task is straightforward- Cost is primary concern- Coordination overhead not worth it- Simple sequential processing sufficient**Q2: How do you handle agent conflicts?****Answer:**Strategies:1. **Hierarchical:** Manager agent makes final decisions2. **Voting:** Majority or weighted voting3. **Consensus:** Agents negotiate until agreement4. **Escalation:** Human breaks ties5. **Priority:** Pre-defined agent hierarchy**Q3: What are the main challenges in multi-agent systems?****Answer:**1. **Coordination Overhead:** More agents = more communication2. **Cost:** Multiple LLM calls expensive3. **Consistency:** Agents may contradict each other4. **Debugging:** Harder to trace errors5. **State Management:** Keeping shared context synchronized**Mitigations:**- Limit agent count (3-5 optimal)- Clear role definitions- Robust communication protocol- Comprehensive logging- Human oversight for critical decisions**Q4: How do CrewAI and AutoGen differ?****Answer:**| Aspect | CrewAI | AutoGen ||--------|--------|---------|| **Paradigm** | Role-based | Conversational || **Focus** | Task delegation | Code execution || **Coordination** | Process types | Turn-taking || **Use Case** | Business workflows | Technical tasks || **Learning Curve** | Lower | Higher || **Flexibility** | Medium | High |**Q5: How do you optimize multi-agent system costs?****Answer:**1. **Agent Pooling:** Reuse agents across tasks2. **Lazy Evaluation:** Only call agents when needed3. **Caching:** Cache agent responses4. **Model Selection:** Use smaller models for simple agents5. **Batching:** Process multiple tasks together6. **Early Termination:** Stop when goal achieved7. **Monitoring:** Track and optimize expensive agents

---# Summary## What We Covered1. ✅ **Multi-Agent Fundamentals** - Architecture, communication, coordination2. ✅ **CrewAI** - Role-based collaboration with sequential/hierarchical processes3. ✅ **AutoGen** - Conversational agents with code execution4. ✅ **Collaboration Patterns** - Hierarchical, sequential, debate, consensus5. ✅ **Production Best Practices** - Monitoring, error handling, optimization6. ✅ **Real-World Applications** - Software dev, content creation## Key Takeaways- **Specialization:** Each agent should have clear, focused role- **Communication:** Manage complexity by limiting agent count- **Process Selection:** Match process type to task requirements- **Monitoring:** Track metrics across all agents- **Cost Optimization:** Use appropriate models and caching## Decision Framework```mermaidflowchart TD    Start[Need Agents?] --> Q1{Multiple<br/>specializations?}    Q1 -->|Yes| Q2{Sequential<br/>dependencies?}    Q1 -->|No| Single[Use Single Agent]        Q2 -->|Yes| Seq[Sequential Process]    Q2 -->|No| Q3{Need manager?}        Q3 -->|Yes| Hier[Hierarchical Process]    Q3 -->|No| Para[Parallel Process]        style Single fill:#c8e6c9    style Seq fill:#fff9c4    style Hier fill:#ffe0b2    style Para fill:#e1f5ff```## Resources- [CrewAI Documentation](https://docs.crewai.com/)- [AutoGen Documentation](https://microsoft.github.io/autogen/)- [Multi-Agent Systems Research](https://arxiv.org/abs/2308.08155)- Next: Research agents, Agentic RAG patterns